In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 메모장
## 변수
* Store :
* Dept : 
* Date : 
* Weekly_Sales : 
* IsHoliday : 

## 가정
* 월초, 월 중순, 월 하순에 따라 판매량에 차이가 있는가?
* 요일에 따라 판매량에 차이가 있는가? -> 날짜 데이터가 금요일만 가지고 있기 때문에 요일에 따른 판매금액 차이는 알수 없음.
단, 금요일 데이터를 가지고 모델링을 하여 제출한뒤, 다른요일 판매량을 예측하는것일수 있음.

* store별, depart별로 판매금액의 차이가 있는가? -> train데이터를 확인한 결과 차이가 있는것으로 보임.

Features변수를 사용하려면 Date값을 기준으로 Join해야할듯.

features에서 2012-10-26 이후의 데이터를 제거한후

# Features

In [ ]:
features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip")
features

In [ ]:
# 날짜형으로 변경
features["Date"] = pd.to_datetime(features["Date"])

In [ ]:
features2 = features[features["Date"] < '2012-10-27']

In [ ]:
features2

# Train

In [ ]:
train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip")
train

In [ ]:
train.groupby("IsHoliday")["Weekly_Sales"].median()

In [ ]:
#pd.merge(train,features2, how = 'left', right_on = 'Date')

In [ ]:
train.dtypes

In [ ]:
# 날짜형으로 변경
train["Date"] = pd.to_datetime(train["Date"])

In [ ]:
# 연도별 데이터를 보기위해 dt.year 추가
train["dateyear"] = train["Date"].dt.year
train["week"] = train["Date"].dt.week

In [ ]:
train3 = train[train["Weekly_Sales"] < 50000]

In [ ]:

plt.figure(figsize = (20,12))
sns.boxplot(train3["week"],train3["Weekly_Sales"])

In [ ]:
# 월별 데이터를 보기 위해 dt.month 추가
train["datemonth"] = train["Date"].dt.month

In [ ]:
# 일별 데이터를 보기 위해 dt.month 추가
train["dateday"] = train["Date"].dt.day

In [ ]:
train["dateyear"].value_counts()

In [ ]:
train["weeknum"] = np.ceil(train["dateday"]/7)

평균과 중간값을 확인한 결과, 연도별 판매금액이 점점 감소하는 경향이 보임

In [ ]:
print(train.groupby("dateyear")["Weekly_Sales"].mean())
train.groupby("dateyear")["Weekly_Sales"].median()

In [ ]:
print(train.groupby("datemonth")["Weekly_Sales"].mean())
train.groupby("datemonth")["Weekly_Sales"].median()

In [ ]:
# 일별 평균과 중간값 확인
print(train.groupby("dateday")["Weekly_Sales"].mean())
train.groupby("dateday")["Weekly_Sales"].median()

데이터를 확인해본 결과 train데이터에는 목요일 데이터만 존재하는것으로 보임

Dept별 판매량의 차이는 415.48에서 19213.48까지 차이가 심한것으로 보이며, store별 판매량도 차이가 있는것으로 보임.

In [ ]:
print(train.groupby("Dept")["Weekly_Sales"].mean())
print(train.groupby("Store")["Weekly_Sales"].mean())

In [ ]:
train = train.merge(stores,on = "Store",how = "left")

In [ ]:
# train 데이터의 Date열 삭제, test와 train의 차원을 맞춰주기 위해 weekly sales를 삭제하고 train2로 저장
train2 = train.drop(["Date","Weekly_Sales"],axis = 1)
train2.head()

# Stores

In [ ]:
stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")
stores.head()

In [ ]:
stores = stores.replace({"A":1,"B":2, "C":3})

In [ ]:
stores.groupby("Type")["Size"].mean()

# Test

In [ ]:
test = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip")
test.head()

In [ ]:
test.dtypes

In [ ]:
test["Date"] = pd.to_datetime(test["Date"])

In [ ]:
# 연도별 데이터를 보기위해 dt.year 추가
test["dateyear"] = test["Date"].dt.year
test["week"] = test["Date"].dt.week

In [ ]:
# 월별 데이터를 보기 위해 dt.month 추가
test["datemonth"] = test["Date"].dt.month

In [ ]:
# 일별 데이터를 보기 위해 dt.month 추가
test["dateday"] = test["Date"].dt.day

In [ ]:
test["dateyear"].value_counts()

test데이터도 확인해본 결과 목요일 데이터만 포함하고있음.

In [ ]:
test["weeknum"] = np.ceil(test["dateday"]/7)

In [ ]:
test = test.merge(stores,on = "Store",how = "left")

In [ ]:
test2 = test.drop(["Date"],axis = 1)
test2.head()

In [ ]:
train["Weekly_Sales"].describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit_transform(train["Weekly_Sales"])

# Modeling

In [ ]:
train["Weekly_Sales"] = np.log(train["Weekly_Sales"])
train = train.fillna(-1)
train.isna().sum()

In [ ]:
print(train2.isna().sum())
test2.isna().sum()

In [ ]:
train["Weekly_Sales"]

In [ ]:
from sklearn.ensemble import RandomForestRegressor 
rf = RandomForestRegressor(n_estimators = 100,n_jobs = 4)
rf.fit(train2,train["Weekly_Sales"])                # train2 데이터로 train의 Weekly_Sales 값을 학습
result = rf.predict(test2)                          # test2를 기반으로 결과값 예측

In [ ]:
sub = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip")
sub.head()
sub["Weekly_Sales"] = result
sub.to_csv("Walmart_sub.csv",index = False)

In [ ]:
sub["Weekly_Sales"] = result
sub.head()

In [ ]:
sub.to_csv("Walmart_sub.csv",index = False)